# Using Producer Mini-App

In [1]:
# System Libraries
import sys, os
sys.path.append("..")
import pandas as pd
import numpy as np
import ast
import pykafka
import mass.KafkaProducer


## logging
import logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger().setLevel(logging.ERROR)
logging.getLogger("py4j").setLevel(logging.ERROR)
logging.getLogger("tornado.application").setLevel(logging.CRITICAL)
logging.getLogger("distributed.utils").setLevel(logging.CRITICAL)


# Pilot-Streaming
import pilot.streaming

import uuid
import time

/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/radical/utils/atfork/stdlib_fixer.py:72: UserWarning: logging handlers already registered.
  warnings.warn('logging handlers already registered.')


# 1. Benchmark Loop

In [ ]:
for num_repeats in range(3):
    for num_producer_nodes in [8,16,32]:
        for num_broker_nodes in [8,16,32]:
    
            kafka_pilot_description1 = {
                "resource":"slurm+ssh://login1.wrangler.tacc.utexas.edu",
                "working_directory": os.path.join('/work/01131/tg804093/wrangler/', "work"),
                "number_cores": 48*num_broker_nodes,
                "project": "TG-MCB090174",
                "queue": "normal",
                "walltime": 159,
                "type":"kafka"
            }
            kafka_pilot = pilot.streaming.PilotComputeService.create_pilot(kafka_pilot_description1)
            kafka_pilot.wait()
            
            dask_pilot_description = {
                "resource":"slurm+ssh://login1.wrangler.tacc.utexas.edu",
                "working_directory": os.path.join('/work/01131/tg804093/wrangler/', "work"),
                "number_cores": 48*num_producer_nodes,
                "project": "TG-MCB090174",
                "queue": "normal",
                "walltime": 159,
                "type":"dask"
            }
            dask_pilot = pilot.streaming.PilotComputeService.create_pilot(dask_pilot_description)
            dask_pilot.wait()
            time.sleep(5)
            for number_parallel_tasks in [4, 8, 16]:
                number_parallel_tasks = num_producer_nodes*number_parallel_tasks
                run_id = str(uuid.uuid1())
                miniapp=mass.KafkaProducer.MiniApp(
                                               dask_scheduler=dask_pilot.get_details()['master_url'],
                                               kafka_zk_hosts=kafka_pilot.get_details()["master_url"],
                                               number_parallel_tasks=number_parallel_tasks,
                                               number_clusters=192,
                                               number_points_per_cluster=52084,
                                               number_points_per_message=5000,
                                               number_dim=3,
                                               number_produces=5,
                                               number_partitions=num_broker_nodes*12,
                                               topic_name="test-"+run_id
                                              )
                miniapp.run()
            
            kafka_pilot.cancel()
            dask_pilot.cancel()

**** Job: 59901 State : Pending
look for configs in: /work/01131/tg804093/wrangler/work/kafka-a27ede94-f942-11e7-bb7e-549f3509766c/config
['broker-5', 'broker-3', 'broker-7', 'broker-1', 'broker-2', 'broker-0', 'broker-6', 'broker-4']
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-a27ede94-f942-11e7-bb7e-549f3509766c/config (Sun Jan 14 09:50:24 2018)
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-a27ede94-f942-11e7-bb7e-549f3509766c/config (Sun Jan 14 09:50:24 2018)
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-a27ede94-f942-11e7-bb7e-549f3509766c/config (Sun Jan 14 09:50:24 2018)
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-a27ede94-f942-11e7-bb7e-549f3509766c/config (Sun Jan 14 09:50:24 2018)
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-a27ede94-f942-11e7-bb7e-549f3509766c/config (Sun Jan 14 09:50:24 2018)
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-a27ede94-f942-11e7-bb7e-549f3509766c/config (Sun Jan 14 09:50:24 2018)
Kafka Con

distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler


**** Job: 59903 State : Pending
look for configs in: /work/01131/tg804093/wrangler/work/kafka-6c0ae0c2-f944-11e7-bb7e-549f3509766c/config
['broker-13', 'broker-5', 'broker-3', 'broker-15', 'broker-7', 'broker-11', 'broker-1', 'broker-12', 'broker-8', 'broker-14', 'broker-2', 'broker-10', 'broker-0', 'broker-6', 'broker-9', 'broker-4']
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-6c0ae0c2-f944-11e7-bb7e-549f3509766c/config (Sun Jan 14 10:03:12 2018)
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-6c0ae0c2-f944-11e7-bb7e-549f3509766c/config (Sun Jan 14 10:03:12 2018)
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-6c0ae0c2-f944-11e7-bb7e-549f3509766c/config (Sun Jan 14 10:03:12 2018)
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-6c0ae0c2-f944-11e7-bb7e-549f3509766c/config (Sun Jan 14 10:03:12 2018)
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-6c0ae0c2-f944-11e7-bb7e-549f3509766c/config (Sun Jan 14 10:03:12 2018)
Kafka Config: /work/01131/tg80409

distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler


**** Job: 59908 State : Pending


distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://129.114.58.132:45635 remote=tcp://c251-124:8786>
distribute

In [3]:
dask_pilot.get_details()

{'master_url': 'tcp://c251-132:8786', 'web_ui_url': 'http://c251-132:8787'}

In [5]:
import distributed
c=distributed.Client("tcp://c251-132:8786")

In [6]:
c.scheduler_info()

{'address': 'tcp://129.114.58.132:8786',
 'id': 'Scheduler-27ca0348-f8fd-4b37-bde9-34244efacc6e',
 'services': {'bokeh': 8787},
 'type': 'Scheduler',
 'workers': {'tcp://129.114.58.132:56688': {'cpu': 10.0,
   'executing': 0,
   'host': '129.114.58.132',
   'in_flight': 0,
   'in_memory': 0,
   'last-seen': 1515899836.044562,
   'local_directory': '/home/01131/tg804093/dask-worker-space/worker-2gVlBS',
   'memory': 103247872,
   'memory_limit': 134778585088,
   'name': 'tcp://129.114.58.132:56688',
   'ncores': 48,
   'num_fds': 23,
   'pid': 106284,
   'read_bytes': 213769.23725120776,
   'ready': 0,
   'services': {'bokeh': 8789, 'nanny': 44744},
   'time': 1515899835.544642,
   'time-delay': 0.0003650188446044922,
   'write_bytes': 213769.23725120776}}}

distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
tornado.application - ERROR - Exception in callback <functools.partial object at 0x2af8d2db62b8>
Traceback (most recent call last):
  File "/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/tornado/ioloop.py", line 605, in _run_callback
    ret = callback()
  File "/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/tornado/ioloop.py", line 626, in _discard_future_result
    future.result()
  File "/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File 

# 2. Mini App Test

Check Kafka Broker

In [ ]:
client = pykafka.KafkaClient(zookeeper_hosts=kafka_details["master_url"])
topic = client.topics['test']
producer = topic.get_sync_producer()
consumer = topic.get_simple_consumer()

In [ ]:
count = 0
number_total_points = 0
number_dimensions = 0
for i in range(100):
    message = consumer.consume(block=False)
    if message is not None:
        data_np = np.array(ast.literal_eval(message.value))
        num_points = data_np.shape[0]
        number_dimensions = data_np.shape[1]
        count =  count + 1
        number_total_points = number_total_points + num_points
    #print "Consumed message: %d, Number Points: %d, Number Dimensions: %d"%\
    #        (count, num_points, number_dimensions)   
        
print("Total Messages: %d, Total Points: %d, Number Dimensions: %d"%(count, number_total_points, number_dimensions))